#### **CUSTOMERS DATA**

In [0]:
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.sql.functions import regexp_replace
from typing import List
from pyspark.sql import DataFrame
from pyspark.sql.window import Window
from delta.tables import DeltaTable
from pyspark.sql.functions import current_timestamp

In [0]:
import os
import sys

In [0]:
from pyspark.sql import DataFrame, Window
from pyspark.sql.functions import col, concat, row_number
from typing import List
from delta.tables import DeltaTable

class transformations:

    def dedup(self, df: DataFrame, dedup_cols: List[str], cdc: str) -> DataFrame:
        """
        Deduplicate a DataFrame by keeping the latest record per key.
        :param df: Input DataFrame
        :param dedup_cols: Columns to deduplicate on
        :param cdc: Column to use for ordering (e.g. last_updated_timestamp)
        :return: Deduplicated DataFrame
        """
        df = df.withColumn("dedupKey", concat(*[col(c) for c in dedup_cols]))
        df = df.withColumn(
            "dedupCounts",
            row_number().over(Window.partitionBy("dedupKey").orderBy(col(cdc).desc()))
        )
        df = df.filter(col("dedupCounts") == 1).drop("dedupKey", "dedupCounts")
        return df
    
    def process_timestamp(self,df):
        df = df.withColumn("process_timestamp",current_timestamp())
        return df
    
    def upsert(self,spark,df, key_cols,table,cdc):
        merge_condition = " AND ".join([f"src.{i} = trg.{i}" for i in key_cols])
        dlt_obj = DeltaTable.forName(spark,f"pysparkdbt.silver.{table}")
        dlt_obj.alias("trg").merge(df.alias("src"),merge_condition)\
                            .whenMatchedUpdateAll(condition = f"src.{cdc} >= trg.{cdc}")\
                            .whenNotMatchedInsertAll()\
                            .execute()

        return "done"
                    

In [0]:
current_dir = os.getcwd()

sys.path.append(current_dir)


In [0]:
df_cust = spark.read.table("pysparkdbt.bronze.customers")

In [0]:
display(df_cust.count())

200

In [0]:
# fetch email domain of customers
df_cust = df_cust.withColumn("domain",split(col('email'),'@')[1])

In [0]:
# clean up phone numbers
df_cust = df_cust.withColumn("phone_number",regexp_replace("phone_number",r"[^0-9]",""))

In [0]:
# concat first and last name to full name and drop the frist anad last name columns
df_cust = df_cust.withColumn("full_name",concat_ws(" ",col('first_name'),col('last_name')))
df_cust = df_cust.drop('first_name','last_name')

In [0]:
cust_obj = transformations()

cust_df_trns = cust_obj.dedup(df_cust,['customer_id'], 'last_updated_timestamp')

In [0]:
df_cust = cust_obj.process_timestamp(cust_df_trns)
display(df_cust)

customer_id email phone_number city signup_date last_updated_timestamp domain full_name process_timestamp 1 azimmerman@ramirez-nelson.com 8117241080 Tiffanyview 2023-12-25 2025-09-15T15:21:05.000Z ramirez-nelson.com Daniel Reed 2025-10-16T10:15:15.514Z 10 vritter@yahoo.com 0019670618100482 Port Jesse 2023-03-12 2025-09-07T23:05:32.000Z yahoo.com Megan Dean 2025-10-16T10:15:15.514Z 100 newmanmelanie@flynn-ross.org 0010575302069 North Charlestown 2022-06-22 2025-09-14T22:47:28.000Z flynn-ross.org Gary Barnes 2025-10-16T10:15:15.514Z 101 hannahwhite@hotmail.com 8882340212190 Lisachester 2020-09-24 2025-09-10T22:45:30.000Z hotmail.com Stacy Thomas 2025-10-16T10:15:15.514Z 102 cunninghamjessica@yahoo.com 0010591315078 Harrisbury 2023-07-29 2025-09-19T21:57:29.000Z yahoo.com Alan Rogers 2025-10-16T10:15:15.514Z 103 sherry55@hernandez.com 12586465512757 Port Michael 2022-06-18 2025-09-09T06:06:57.000Z hernandez.com Stephen Sanders 2025-10-16T10:15:15.514Z 104 moralessandy@arnold-robinson.com 0018634267566120 Lake Kelly 2020-11-07 2025-09-17T02:22:22.000Z arnold-robinson.com Duane Bennett 2025-10-16T10:15:15.514Z 105 matthewsbenjamin@lloyd.com 87710366965279 South Wyattfort 2021-04-11 2025-08-25T11:18:42.000Z lloyd.com Jasmin Patterson 2025-10-16T10:15:15.514Z 106 tyleryoung@gross.org 41127166452578 Seanview 2024-01-05 2025-08-25T04:33:25.000Z gross.org Kara Williams 2025-10-16T10:15:15.514Z 107 brandyphillips@hotmail.com 17891625036 Melissabury 2021-06-15 2025-09-14T13:56:22.000Z hotmail.com Martin Williams 2025-10-16T10:15:15.514Z 108 yodonnell@gmail.com 6792566118 Port Brandon 2024-10-18 2025-09-16T17:58:10.000Z gmail.com Julian Bell 2025-10-16T10:15:15.514Z 109 zweaver@hotmail.com 0018387122136 Jacquelineside 2021-05-11 2025-09-01T22:46:52.000Z hotmail.com Sharon Reed 2025-10-16T10:15:15.514Z 11 kristen04@hotmail.com 3920669676001 South Stephanieberg 2022-04-28 2025-09-07T03:15:02.000Z hotmail.com Barbara Anderson 2025-10-16T10:15:15.514Z 110 qevans@hotmail.com 0475263467475 New Susan 2022-06-22 2025-09-19T09:00:30.000Z hotmail.com Christine Smith 2025-10-16T10:15:15.514Z 111 thomas64@yahoo.com 59379630218361 New James 2025-07-08 2025-08-26T05:04:41.000Z yahoo.com Joseph Hampton 2025-10-16T10:15:15.514Z 112 brandontanner@maldonado-brown.com 1088911686 North Martinfort 2022-03-27 2025-09-11T17:51:51.000Z maldonado-brown.com Joshua Peterson 2025-10-16T10:15:15.514Z 113 amy19@hotmail.com 546921808799627 North Brittanyshire 2022-12-04 2025-09-08T04:59:57.000Z hotmail.com Kari Bond 2025-10-16T10:15:15.514Z 114 stewartkevin@reed.com 001225555519882023 Port Amandabury 2025-03-29 2025-08-31T20:03:20.000Z reed.com Melissa Johnson 2025-10-16T10:15:15.514Z 115 hornronald@hall-gutierrez.com 767288145234310 Gavinfort 2025-01-12 2025-09-02T02:33:28.000Z hall-gutierrez.com Thomas Elliott 2025-10-16T10:15:15.514Z 116 parrishrichard@yahoo.com 4505435605 Huntborough 2023-02-03 2025-09-02T22:29:59.000Z yahoo.com Jennifer Grimes 2025-10-16T10:15:15.514Z 117 madison07@mendoza.com 5261954761 West Andrew 2024-10-07 2025-09-01T21:01:28.000Z mendoza.com Christopher Jones 2025-10-16T10:15:15.514Z 118 mosstroy@gmail.com 09154369942249 Payneshire 2024-04-26 2025-08-27T23:32:46.000Z gmail.com Joshua Sutton 2025-10-16T10:15:15.514Z 119 michaelthornton@raymond-clarke.com 8685503172 East Tonyshire 2021-10-30 2025-08-29T06:52:55.000Z raymond-clarke.com Joshua Bailey 2025-10-16T10:15:15.514Z 12 wilsonlaura@lewis.com 1648097691462270 Andrewton 2023-11-10 2025-09-01T23:32:42.000Z lewis.com Laura Roman 2025-10-16T10:15:15.514Z 120 millertammy@howard-johnson.info 9663989624869 South Jessicaville 2020-12-10 2025-09-01T04:47:14.000Z howard-johnson.info Daniel Coleman 2025-10-16T10:15:15.514Z 121 norma62@luna.com 0010171858293666 South Jamestown 2021-12-30 2025-09-13T18:50:00.000Z luna.com Richard Thompson 2025-10-16T10:15:15.514Z 122 williamhicks@hotmail.com 3871896972098 Zacharyfort 2021-05-03 2025-09-18T19:10:02.000Z hotmail.com Elizabeth Jones 2025-10-16T10:15:15.514

In [0]:
if not spark.catalog.tableExists("pysparkdbt.silver.customers"):

    df_cust.write.format("delta")\
        .mode("append")\
        .saveAsTable("pysparkdbt.silver.customers")

else:
    cust_obj.upsert(spark,df_cust,['customer_id'],'customers','last_updated_timestamp')

In [0]:
%sql
-- querry the table
SELECT COUNT(*) FROM pysparkdbt.silver.customers

COUNT(*) 200

#### **DRIVERS DATA**

In [0]:
df_driver = spark.read.table("pysparkdbt.bronze.drivers")
display(df_driver)

driver_id first_name last_name phone_number vehicle_id driver_rating city last_updated_timestamp 1 Latasha Lopez 262-924-2955x590 1 4.7 East Dorothy 2025-08-25T06:36:26.000Z 2 Alan Wiley 0967969634 2 3.98 West Susan 2025-09-14T00:44:57.000Z 3 James Taylor 424-614-1847 3 3.66 Mcintoshton 2025-08-26T22:28:17.000Z 4 Theresa Benson 617-017-0101x91777 4 3.86 North Courtneychester 2025-09-01T11:40:55.000Z 5 Karen Jensen 611-060-5683 5 4.87 Brownburgh 2025-09-04T16:35:04.000Z 6 Debra Smith 556.480.9096x439 6 4.26 Port Williamland 2025-08-31T14:31:37.000Z 7 Justin Peters +1-798-568-6952x9778 7 3.67 West Erinborough 2025-09-17T05:57:45.000Z 8 Todd Young 706.321.8390x08097 8 4.9 Lake Stephen 2025-08-26T06:45:51.000Z 9 Mary Young (172)791-0504x5499 9 4.5 West Lindsey 2025-08-27T13:04:32.000Z 10 Jacob Mack (509)613-4480 10 4.04 Lauraland 2025-09-09T05:50:23.000Z 11 Matthew Barajas (143)077-8973 11 3.58 Lake Stephenview 2025-08-20T18:43:00.000Z 12 Kimberly Sampson 694-869-6563 12 4.91 New Michaelview 2025-08-25T18:14:33.000Z 13 David Silva 3263292468 13 4.52 Strongfort 2025-09-09T09:45:17.000Z 14 Sean Christensen 719-249-3764 14 4.83 South Tina 2025-09-18T20:52:07.000Z 15 Vanessa Williams (205)568-3393 15 4.38 Petersenfurt 2025-09-01T07:01:10.000Z 16 Ryan Henderson 001-267-031-5560 16 3.53 New Gabrielleside 2025-08-22T03:44:26.000Z 17 Alfred Clayton 644.168.2978x787 17 3.83 Sydneystad 2025-08-30T15:09:25.000Z 18 Lisa Duarte 015-912-0292x438 18 4.12 New Michaelshire 2025-09-13T08:53:24.000Z 19 Gregory Stevenson 390-659-6818 19 4.36 Cynthiachester 2025-09-10T16:50:52.000Z 20 Jon Cole (312)494-1187x781 20 3.86 Janetville 2025-09-13T11:31:40.000Z 21 Gina Douglas (541)441-1442 21 4.26 Sanchezville 2025-09-03T05:20:45.000Z 22 Melissa Erickson 7192372004 22 4.59 South Arthurhaven 2025-08-30T21:59:16.000Z 23 Daniel Hill +1-577-556-3399x04501 23 4.88 East Katherine 2025-09-16T05:49:17.000Z 24 John Wise (799)900-1244x5082 24 3.55 West Mark 2025-09-06T08:42:49.000Z 25 Mark Smith (021)487-3147x3928 25 4.35 Lake Jeffreyburgh 2025-08-27T23:53:03.000Z 26 Shawn Taylor 001-748-759-7040x959 26 3.99 Woodville 2025-09-13T18:41:49.000Z 27 Antonio Armstrong 001-032-664-7995 27 4.43 New Jeremymouth 2025-08-21T05:39:11.000Z 28 Monica Ruiz 001-231-920-7076x0529 28 4.89 West Heidi 2025-08-21T20:36:05.000Z 29 Kristen Brewer 404-585-9239x5636 29 3.57 Sarahborough 2025-09-09T03:49:41.000Z 30 Stephanie Perry 010-068-4248x5965 30 3.82 Clementsmouth 2025-08-23T16:17:51.000Z 31 Margaret Mejia 001-572-346-9122 31 3.88 South Josephmouth 2025-09-07T11:27:44.000Z 32 Carrie Cook 920.128.5027 32 4.39 North Jesseside 2025-08-26T16:05:18.000Z 33 Julia Ramirez 514.664.1295 33 4.02 Natalieshire 2025-08-24T16:29:13.000Z 34 Michelle Wallace +1-396-982-1652x448 34 3.96 Micheleside 2025-09-01T17:06:10.000Z 35 Christopher Foley 015-274-9057x8050 35 4.66 East Brendatown 2025-08-30T04:09:05.000Z 36 Jessica Bradley +1-437-108-7082 36 4.54 New Annaberg 2025-09-15T08:39:54.000Z 37 Jennifer Thompson 979.323.5029 37 4.27 Smithstad 2025-09-14T19:21:18.000Z 38 Curtis Smith (316)479-3205 38 4.73 South Johnmouth 2025-09-18T07:04:18.000Z 39 Karen Williamson (503)737-0807x0440 39 3.86 North Shellyberg 2025-08-28T11:19:44.000Z 40 Sarah Ross 739.325.1580 40 3.63 Charlesborough 2025-09-13T02:43:48.000Z 41 Anna Wells +1-722-457-9572x75757 41 3.8 Christianshire 2025-09-19T19:53:28.000Z 42 Allison Wheeler +1-538-876-3403x44318 42 4.03 New Ryanton 2025-08-24T04:42:04.000Z 43 Nicholas Williamson 414-666-7430 43 4.87 Ericmouth 2025-09-20T01:25:54.000Z 44 Jared Terry 001-244-056-5144x53788 44 4.45 Lake Melissa 2025-09-14T22:58:31.000Z 45 Howard Hickman 212-434-7041x14381 45 4.62 Deborahhaven 2025-08-21T15:18:53.000Z 46 Michael Smith (075)616-0777x6099 46 3.51 Howardchester 2025-08-28T23:55:17.000Z 47 Sherry Hartman 001-895-841-5328 47 4.2 Tranport 2025-08-24T05:33:22.000Z 48 Sarah Simpson 202.288.7792x90872 48 4.75 Harveymouth 2025-09-15T04:50:36.000Z 49 Tiffany Marks 970-585-4006 49 4.63 Lyonsland 2025-09-14

In [0]:
# clean up phone numbers
df_driver = df_driver.withColumn("phone_number",regexp_replace("phone_number",r"[^0-9]",""))

In [0]:
# concat first and last name to full name and drop the frist anad last name columns
df_driver = df_driver.withColumn("full_name",concat_ws(" ",col('first_name'),col('last_name')))
df_driver = df_driver.drop('first_name','last_name')

In [0]:
driver_obj = transformations()

In [0]:
# deduplication
df_driver = driver_obj.dedup(df_driver,['driver_id'],'last_updated_timestamp')

In [0]:
#timestamp
df_driver = driver_obj.process_timestamp(df_driver)

In [0]:
#apply upsert
if not spark.catalog.tableExists("pysparkdbt.silver.drivers"):

    df_driver.write.format("delta")\
            .mode("append")\
            .saveAsTable("pysparkdbt.silver.drivers")

else:
    driver_obj.upsert(spark,df_driver,['driver_id'],'drivers','last_updated_timestamp')

In [0]:
%sql
SELECT COUNT(*) FROM pysparkdbt.silver.drivers

COUNT(*) 50

#### **LOCATIONS DATA**

In [0]:
df_loc = spark.read.table("pysparkdbt.bronze.locations")
display(df_loc)

location_id city state country latitude longitude last_updated_timestamp 1 Lake Davidport Hawaii Dominica 34.5682 151.8097 2025-09-03T13:37:24.000Z 2 Mccarthybury Minnesota Dominican Republic 71.6928 -162.2836 2025-09-10T10:54:18.000Z 3 Bellhaven Arkansas Japan -53.3075 69.1475 2025-09-17T06:52:51.000Z 4 Moorechester New Hampshire Togo 77.4281 57.0052 2025-08-27T19:04:27.000Z 5 Glennview North Carolina Gambia 41.1122 -162.4836 2025-08-20T17:31:06.000Z 6 Davilaville Arizona Antarctica (the territory South of 60 deg S) 64.3413 -3.912 2025-09-05T13:22:58.000Z 7 Shawnfurt New Jersey Lithuania 71.1198 -143.6791 2025-09-03T20:35:10.000Z 8 Bradleytown Florida Monaco 30.0895 88.091 2025-08-29T19:45:05.000Z 9 East Miguel Maryland French Guiana -52.9274 -56.9343 2025-08-27T23:43:41.000Z 10 Masonside Wyoming Saint Lucia 84.9905 81.5852 2025-09-06T20:25:27.000Z 11 Gregorytown Connecticut Venezuela -20.4441 -110.9417 2025-09-19T15:34:25.000Z 12 Woodstown North Dakota Seychelles -55.2071 54.4873 2025-09-16T02:31:19.000Z 13 North Susanstad Wisconsin Tunisia 29.7243 -32.3773 2025-08-22T18:01:13.000Z 14 North Jodimouth Nevada Togo -56.7022 -94.9916 2025-08-26T08:31:22.000Z 15 Stevensstad Missouri Italy -66.9045 133.5157 2025-08-26T16:59:13.000Z 16 New Josephburgh Nebraska Lesotho -50.8801 1.9203 2025-09-15T17:26:33.000Z 17 Zacharyport Utah Peru -63.2489 63.0064 2025-08-30T22:04:52.000Z 18 South Cynthia Oklahoma Cayman Islands -8.8718 -27.5926 2025-09-18T16:00:47.000Z 19 Orrburgh Louisiana Syrian Arab Republic -88.1953 86.5364 2025-08-26T17:21:40.000Z 20 South Amandamouth Pennsylvania Grenada 34.9326 -125.7613 2025-09-09T02:09:46.000Z 21 North Jamie New York Cape Verde 13.0148 108.2596 2025-08-22T18:31:57.000Z 22 Daviesview North Dakota Tunisia -71.9351 133.3636 2025-08-22T13:57:32.000Z 23 South Karenstad North Dakota Ecuador 24.7551 -171.1737 2025-08-29T23:09:41.000Z 24 Robertbury Louisiana Vietnam -88.331 150.8571 2025-08-24T09:02:23.000Z 25 Davidbury West Virginia Marshall Islands 14.1393 6.7921 2025-08-24T01:44:16.000Z 26 Jeremiahchester Louisiana Barbados -20.7269 -40.0347 2025-09-11T02:19:04.000Z 27 Jimenezview New York Tokelau -13.977 -130.9149 2025-09-09T09:26:45.000Z 28 Cummingsmouth Delaware Palestinian Territory 1.1331 81.7435 2025-09-15T17:06:45.000Z 29 North Christophermouth South Carolina Australia 44.462 -18.8701 2025-09-06T08:56:51.000Z 30 Nicholsonshire Washington Ghana -77.731 125.1139 2025-08-30T04:54:08.000Z 31 West Lacey New Mexico Azerbaijan -76.3979 58.5535 2025-08-22T04:30:25.000Z 32 Pattonmouth Illinois Guinea-Bissau -17.5733 33.7067 2025-09-16T06:48:31.000Z 33 Perezhaven Utah Macao 12.0753 -147.9233 2025-08-23T23:06:22.000Z 34 East Stephen Georgia Timor-Leste -66.4608 -104.7628 2025-09-14T16:24:12.000Z 35 Josephstad Connecticut Swaziland 34.1588 -1.8357 2025-09-15T08:23:42.000Z 36 Bensonport South Dakota Cocos (Keeling) Islands 40.3832 -108.2375 2025-08-27T01:32:31.000Z 37 East Ericaborough Alaska Saint Kitts and Nevis -44.2057 -109.9523 2025-09-14T13:43:11.000Z 38 Joshuaborough New York Senegal 39.6279 -91.3595 2025-08-31T01:54:05.000Z 39 Jasonborough Illinois Mexico -79.7627 -52.5449 2025-09-16T15:08:16.000Z 40 North Justin Nebraska Trinidad and Tobago 60.3973 123.0022 2025-09-08T07:08:23.000Z 41 East Heidi Florida Central African Republic -52.1685 146.0277 2025-09-19T12:20:16.000Z 42 Ericport Mississippi Kazakhstan -56.0771 143.8596 2025-09-11T12:34:31.000Z 43 Craigland New York Saint Vincent and the Grenadines 5.3862 144.7674 2025-09-15T02:07:48.000Z 44 New Eric Alabama Switzerland 62.795 113.7292 2025-09-15T09:55:44.000Z 45 North Williamport Kansas Palau 12.7332 135.9089 2025-09-09T19:55:53.000Z 46 Rickyton Oregon Togo 13.3392 68.3422 2025-09-20T03:44:42.000Z 47 South Regina Missouri Brunei Darussalam 26.6862 169.5709 2025-08-27T08:24:37.000Z 48 West Lawrenceberg Nebraska Azerbaijan -76.9552 -66.4315 2025-09-20T01:33:28.000Z 49 Steventon Indiana Ghana -5.9245 22.5731 2025-08-22T06:50:48.000Z 50 Port Brenda Wiscons

In [0]:
# Apply transformations
loc_obj = transformations()
df_loc = loc_obj.dedup(df_loc, ['location_id'], 'last_updated_timestamp')
df_loc = loc_obj.process_timestamp(df_loc)

In [0]:
if not spark.catalog.tableExists("pysparkdbt.silver.locations"):
    df_loc.write.format("delta")\
        .mode("append")\
        .saveAsTable("pysparkdbt.silver.locations")
else:
    loc_obj.upsert(spark, df_loc, ['location_id'], 'locations', 'last_updated_timestamp')

In [0]:
%sql
SELECT COUNT(*) FROM pysparkdbt.silver.locations

COUNT(*) 50

#### **PAYMENTS DATA**

In [0]:
df_pay = spark.read.table("pysparkdbt.bronze.payments")
display(df_pay)

payment_id trip_id customer_id payment_method payment_status amount transaction_time last_updated_timestamp 1 274 126 Cash Success 38.15 2025-09-17T13:00:12.000Z 2025-08-30T13:40:53.000Z 2 676 131 Cash Success 52.07 2025-08-14T13:00:12.000Z 2025-09-08T18:21:05.000Z 3 919 132 Card Failed 55.5 2025-07-27T13:00:12.000Z 2025-08-21T20:24:08.000Z 4 247 34 Wallet Pending 28.78 2025-07-27T13:00:12.000Z 2025-08-27T15:03:09.000Z 5 386 62 Card Failed 55.02 2025-08-01T13:00:12.000Z 2025-09-08T23:15:06.000Z 6 834 78 Wallet Success 27.43 2025-08-25T13:00:12.000Z 2025-09-20T09:03:05.000Z 7 348 144 Wallet Failed 74.94 2025-08-26T13:00:12.000Z 2025-08-30T19:11:26.000Z 8 558 82 Card Failed 19.58 2025-09-09T13:00:12.000Z 2025-09-06T05:53:26.000Z 9 260 151 Card Pending 24.4 2025-07-23T13:00:12.000Z 2025-09-02T17:43:14.000Z 10 133 70 Cash Failed 69.99 2025-07-31T13:00:12.000Z 2025-09-11T10:30:16.000Z 11 616 33 Card Success 73.31 2025-07-25T13:00:12.000Z 2025-08-24T09:09:29.000Z 12 333 92 Cash Success 95.29 2025-07-23T13:00:12.000Z 2025-09-11T03:49:52.000Z 13 697 33 Cash Success 94.39 2025-07-04T13:00:12.000Z 2025-09-04T22:11:28.000Z 14 680 121 Wallet Pending 46.3 2025-06-24T13:00:12.000Z 2025-09-14T20:00:33.000Z 15 235 112 Wallet Failed 22.54 2025-09-10T13:00:12.000Z 2025-08-25T22:37:16.000Z 16 45 81 Cash Failed 16.34 2025-08-14T13:00:12.000Z 2025-09-16T08:08:07.000Z 17 82 124 Card Failed 11.26 2025-08-05T13:00:12.000Z 2025-09-15T00:36:40.000Z 18 303 134 Cash Success 39.59 2025-08-03T13:00:12.000Z 2025-09-13T08:28:30.000Z 19 101 91 Card Success 10.41 2025-07-11T13:00:12.000Z 2025-09-08T01:33:11.000Z 20 978 50 Cash Failed 20.21 2025-08-06T13:00:12.000Z 2025-09-12T11:00:42.000Z 21 794 28 Cash Failed 95.71 2025-08-15T13:00:12.000Z 2025-08-22T14:55:10.000Z 22 512 3 Card Success 67.49 2025-07-01T13:00:12.000Z 2025-09-06T14:34:23.000Z 23 47 2 Cash Success 13.2 2025-08-06T13:00:12.000Z 2025-09-19T19:21:27.000Z 24 308 40 Cash Pending 32.8 2025-06-22T13:00:12.000Z 2025-08-20T23:46:31.000Z 25 571 143 Card Pending 54.29 2025-07-31T13:00:12.000Z 2025-09-04T15:55:07.000Z 26 705 116 Wallet Pending 90.15 2025-07-31T13:00:12.000Z 2025-09-07T17:45:10.000Z 27 8 83 Cash Success 69.8 2025-07-19T13:00:12.000Z 2025-09-18T23:10:19.000Z 28 804 96 Wallet Failed 41.39 2025-07-01T13:00:12.000Z 2025-08-27T12:41:51.000Z 29 874 53 Cash Failed 26.44 2025-07-29T13:00:12.000Z 2025-08-29T07:19:07.000Z 30 817 68 Wallet Pending 93.11 2025-08-29T13:00:12.000Z 2025-09-13T22:11:06.000Z 31 798 66 Card Pending 55.66 2025-08-07T13:00:12.000Z 2025-09-13T04:17:49.000Z 32 289 4 Cash Success 60.23 2025-09-14T13:00:12.000Z 2025-09-01T17:19:56.000Z 33 614 7 Wallet Pending 96.09 2025-07-30T13:00:12.000Z 2025-08-30T12:56:09.000Z 34 183 90 Card Pending 8.86 2025-08-23T13:00:12.000Z 2025-09-11T04:37:35.000Z 35 154 148 Cash Success 41.44 2025-08-20T13:00:12.000Z 2025-09-02T17:08:51.000Z 36 230 37 Wallet Success 21.06 2025-09-15T13:00:12.000Z 2025-08-28T08:09:57.000Z 37 54 130 Wallet Pending 8.78 2025-09-19T13:00:12.000Z 2025-09-02T00:26:03.000Z 38 426 67 Card Pending 31.83 2025-08-29T13:00:12.000Z 2025-09-05T19:18:30.000Z 39 578 133 Cash Failed 51.01 2025-08-19T13:00:12.000Z 2025-09-18T21:09:56.000Z 40 331 129 Cash Pending 62.41 2025-07-28T13:00:12.000Z 2025-08-23T01:48:39.000Z 41 440 145 Cash Failed 95.43 2025-08-17T13:00:12.000Z 2025-08-26T06:56:29.000Z 42 859 162 Cash Success 50.2 2025-06-27T13:00:12.000Z 2025-09-14T11:21:07.000Z 43 469 92 Wallet Failed 18.99 2025-09-16T13:00:12.000Z 2025-08-24T01:27:19.000Z 44 29 74 Card Failed 27.57 2025-06-22T13:00:12.000Z 2025-09-01T13:41:52.000Z 45 299 22 Card Pending 61.54 2025-09-06T13:00:12.000Z 2025-09-14T15:20:20.000Z 46 320 113 Card Pending 98.83 2025-09-17T13:00:12.000Z 2025-09-13T20:14:30.000Z 47 249 93 Cash Success 61.56 2025-09-02T13:00:12.000Z 2025-08-23T17:15:26.000Z 48 271 20 Card Success 36.53 2025-08-16T13:00:12.000Z 2025-08-30T13:02:38.000Z 49 752 18 Card Success 70.94 2025-08-07T13:00:12.000Z 2025-09-02T02:49:26.000Z 50 536 63 Card Pending 4

In [0]:
df_pay = df_pay.withColumn("online_payment_status",
            when(  ((col("payment_method") == "Card") & (col("payment_status") == 'Success')),"online-success" )
            .when(  ((col("payment_method") == "Card") & (col("payment_status") == 'Failed')),"online-failure" )
            .when(  ((col("payment_method") == "Cash") & (col("payment_status") == 'pending')),"online-pending" )
            .otherwise("offline")
            )
display(df_pay)

payment_id trip_id customer_id payment_method payment_status amount transaction_time last_updated_timestamp online_payment_status 1 274 126 Cash Success 38.15 2025-09-17T13:00:12.000Z 2025-08-30T13:40:53.000Z offline 2 676 131 Cash Success 52.07 2025-08-14T13:00:12.000Z 2025-09-08T18:21:05.000Z offline 3 919 132 Card Failed 55.5 2025-07-27T13:00:12.000Z 2025-08-21T20:24:08.000Z online-failure 4 247 34 Wallet Pending 28.78 2025-07-27T13:00:12.000Z 2025-08-27T15:03:09.000Z offline 5 386 62 Card Failed 55.02 2025-08-01T13:00:12.000Z 2025-09-08T23:15:06.000Z online-failure 6 834 78 Wallet Success 27.43 2025-08-25T13:00:12.000Z 2025-09-20T09:03:05.000Z offline 7 348 144 Wallet Failed 74.94 2025-08-26T13:00:12.000Z 2025-08-30T19:11:26.000Z offline 8 558 82 Card Failed 19.58 2025-09-09T13:00:12.000Z 2025-09-06T05:53:26.000Z online-failure 9 260 151 Card Pending 24.4 2025-07-23T13:00:12.000Z 2025-09-02T17:43:14.000Z offline 10 133 70 Cash Failed 69.99 2025-07-31T13:00:12.000Z 2025-09-11T10:30:16.000Z offline 11 616 33 Card Success 73.31 2025-07-25T13:00:12.000Z 2025-08-24T09:09:29.000Z online-success 12 333 92 Cash Success 95.29 2025-07-23T13:00:12.000Z 2025-09-11T03:49:52.000Z offline 13 697 33 Cash Success 94.39 2025-07-04T13:00:12.000Z 2025-09-04T22:11:28.000Z offline 14 680 121 Wallet Pending 46.3 2025-06-24T13:00:12.000Z 2025-09-14T20:00:33.000Z offline 15 235 112 Wallet Failed 22.54 2025-09-10T13:00:12.000Z 2025-08-25T22:37:16.000Z offline 16 45 81 Cash Failed 16.34 2025-08-14T13:00:12.000Z 2025-09-16T08:08:07.000Z offline 17 82 124 Card Failed 11.26 2025-08-05T13:00:12.000Z 2025-09-15T00:36:40.000Z online-failure 18 303 134 Cash Success 39.59 2025-08-03T13:00:12.000Z 2025-09-13T08:28:30.000Z offline 19 101 91 Card Success 10.41 2025-07-11T13:00:12.000Z 2025-09-08T01:33:11.000Z online-success 20 978 50 Cash Failed 20.21 2025-08-06T13:00:12.000Z 2025-09-12T11:00:42.000Z offline 21 794 28 Cash Failed 95.71 2025-08-15T13:00:12.000Z 2025-08-22T14:55:10.000Z offline 22 512 3 Card Success 67.49 2025-07-01T13:00:12.000Z 2025-09-06T14:34:23.000Z online-success 23 47 2 Cash Success 13.2 2025-08-06T13:00:12.000Z 2025-09-19T19:21:27.000Z offline 24 308 40 Cash Pending 32.8 2025-06-22T13:00:12.000Z 2025-08-20T23:46:31.000Z offline 25 571 143 Card Pending 54.29 2025-07-31T13:00:12.000Z 2025-09-04T15:55:07.000Z offline 26 705 116 Wallet Pending 90.15 2025-07-31T13:00:12.000Z 2025-09-07T17:45:10.000Z offline 27 8 83 Cash Success 69.8 2025-07-19T13:00:12.000Z 2025-09-18T23:10:19.000Z offline 28 804 96 Wallet Failed 41.39 2025-07-01T13:00:12.000Z 2025-08-27T12:41:51.000Z offline 29 874 53 Cash Failed 26.44 2025-07-29T13:00:12.000Z 2025-08-29T07:19:07.000Z offline 30 817 68 Wallet Pending 93.11 2025-08-29T13:00:12.000Z 2025-09-13T22:11:06.000Z offline 31 798 66 Card Pending 55.66 2025-08-07T13:00:12.000Z 2025-09-13T04:17:49.000Z offline 32 289 4 Cash Success 60.23 2025-09-14T13:00:12.000Z 2025-09-01T17:19:56.000Z offline 33 614 7 Wallet Pending 96.09 2025-07-30T13:00:12.000Z 2025-08-30T12:56:09.000Z offline 34 183 90 Card Pending 8.86 2025-08-23T13:00:12.000Z 2025-09-11T04:37:35.000Z offline 35 154 148 Cash Success 41.44 2025-08-20T13:00:12.000Z 2025-09-02T17:08:51.000Z offline 36 230 37 Wallet Success 21.06 2025-09-15T13:00:12.000Z 2025-08-28T08:09:57.000Z offline 37 54 130 Wallet Pending 8.78 2025-09-19T13:00:12.000Z 2025-09-02T00:26:03.000Z offline 38 426 67 Card Pending 31.83 2025-08-29T13:00:12.000Z 2025-09-05T19:18:30.000Z offline 39 578 133 Cash Failed 51.01 2025-08-19T13:00:12.000Z 2025-09-18T21:09:56.000Z offline 40 331 129 Cash Pending 62.41 2025-07-28T13:00:12.000Z 2025-08-23T01:48:39.000Z offline 41 440 145 Cash Failed 95.43 2025-08-17T13:00:12.000Z 2025-08-26T06:56:29.000Z offline 42 859 162 Cash Success 50.2 2025-06-27T13:00:12.000Z 2025-09-14T11:21:07.000Z offline 43 469 92 Wallet Failed 18.99 2025-09-16T13:00:12.000Z 2025-08-24T01:27:19.000Z offline 44 29 74 Card Failed 27.57 2025-06-22T13:00:12.000Z 2025-09-01T13:41:52.000Z onlin

In [0]:
payment_obj = transformations()
df_pay = payment_obj.dedup(df_pay,['payment_id'],'last_updated_timestamp')
df_pay = payment_obj.process_timestamp(df_pay)

In [0]:
if not spark.catalog.tableExists("pysparkdbt.silver.payments"):
    df_pay.write.format("delta")\
        .mode("append")\
        .saveAsTable("pysparkdbt.silver.payments")
else:
    payment_obj.upsert(spark, df_pay, ['payment_id'], 'payments', 'last_updated_timestamp')

In [0]:
%sql
SELECT COUNT(*) FROM pysparkdbt.silver.payments

COUNT(*) 1000

#### **VEHICLES DATA**

In [0]:
df_veh = spark.read.table("pysparkdbt.bronze.vehicles")
display(df_veh)

vehicle_id license_plate model make year vehicle_type last_updated_timestamp 1 NXT-8646 Message Francis, Smith and Lee 2023 Hatchback 2025-09-02T06:05:20.000Z 2 03S R43 Region Lawson Group 2017 Sedan 2025-08-31T05:55:09.000Z 3 SKO H06 Prepare Moreno, Ruiz and Barker 2023 Luxury 2025-08-30T05:07:29.000Z 4 5R235 Pattern Welch, Martinez and Hendricks 2019 Van 2025-09-09T05:52:10.000Z 5 925A Process Rivera-Anderson 2014 Hatchback 2025-08-27T11:42:16.000Z 6 6-1130V On Brown Ltd 2014 SUV 2025-09-06T06:37:52.000Z 7 L25-TWN Plan Gonzalez, Rios and Rios 2020 Sedan 2025-09-15T13:14:48.000Z 8 GWY 958 Month Smith, Mckenzie and Bullock 2017 Sedan 2025-09-11T16:20:12.000Z 9 4FG 919 Speak Rice, Barnes and Hernandez 2019 SUV 2025-09-11T02:43:13.000Z 10 6FD 648 Religious Schwartz and Sons 2017 SUV 2025-09-11T19:10:01.000Z 11 0N 8334L Name Evans-King 2010 SUV 2025-09-10T19:40:04.000Z 12 3YH 743 So Martinez and Sons 2011 Sedan 2025-09-03T19:53:15.000Z 13 ULA0960 Event Reed Inc 2010 Sedan 2025-08-31T05:38:52.000Z 14 982MCB Road Chavez Group 2014 Luxury 2025-08-25T11:03:53.000Z 15 0I 6R0MOA Item Robbins LLC 2015 Sedan 2025-08-20T16:17:49.000Z 16 6WJ63 Store Johnson-Prince 2017 Van 2025-09-07T09:41:02.000Z 17 542-VGUN East Perez, Lee and Harris 2023 Sedan 2025-09-02T18:06:22.000Z 18 RLY 8125 Low Luna, Bryant and Rhodes 2019 Luxury 2025-09-19T17:46:03.000Z 19 JV-9841 Now Lawrence Inc 2019 SUV 2025-08-24T06:13:32.000Z 20 9U 7I8WLE Own Webb, Fowler and Murray 2018 Luxury 2025-08-29T14:59:19.000Z 21 JYI-465 Rule Allen and Sons 2014 Sedan 2025-09-02T02:34:55.000Z 22 53-FE88 Face Stephenson-Robinson 2012 Hatchback 2025-08-30T05:46:20.000Z 23 5T WT867 Information Collins PLC 2013 Van 2025-09-09T17:04:46.000Z 24 96-4009B Question Walker PLC 2023 Van 2025-08-28T20:18:45.000Z 25 ZMM 744 Upon Arellano-Williams 2016 Luxury 2025-08-27T22:33:12.000Z 26 KE 35607 Go Owens PLC 2021 Hatchback 2025-09-07T07:47:40.000Z 27 602 CUA Blood Baker and Sons 2016 Van 2025-09-13T05:32:48.000Z 28 UCT-785 Build Sampson-Harrison 2010 Luxury 2025-08-21T12:41:00.000Z 29 7W 2C1MWN But Garcia-Craig 2015 Hatchback 2025-09-12T05:34:31.000Z 30 KBS5092 Care Carter and Sons 2022 Hatchback 2025-08-27T04:57:47.000Z 31 8ESI316 Enter Martin and Sons 2023 Sedan 2025-09-09T11:06:50.000Z 32 VQC N88 Consumer Boyd-Cruz 2016 Hatchback 2025-09-20T01:23:10.000Z 33 4596 PJ Sort Butler-Salinas 2019 Sedan 2025-09-20T08:41:13.000Z 34 MQL 012 Kind Collier-Baker 2016 SUV 2025-09-09T16:45:34.000Z 35 487 MVB South Gregory, Sanchez and Lee 2019 Sedan 2025-09-09T11:03:29.000Z 36 6TU5286 Help Summers-Foster 2022 Van 2025-09-03T16:13:47.000Z 37 393 0841 Per Palmer Ltd 2020 SUV 2025-09-07T09:56:14.000Z 38 1W 5D5FFZ Mission Hill PLC 2015 Sedan 2025-09-16T17:24:54.000Z 39 2V 58100 Well Woods-Parks 2018 Sedan 2025-08-22T09:11:03.000Z 40 FGE 827 Expert Parsons Inc 2011 Hatchback 2025-09-16T16:04:50.000Z 41 LO 3414 Drop Cole Ltd 2021 Luxury 2025-09-14T11:29:26.000Z 42 LB-9270 Government Gutierrez, Moreno and Davis 2023 SUV 2025-08-27T09:52:46.000Z 43 6ID56 Call Lewis PLC 2018 SUV 2025-09-12T18:28:24.000Z 44 094 TEH Court Warner LLC 2020 SUV 2025-08-23T13:57:59.000Z 45 AWZ2518 With Roth-Rich 2016 Hatchback 2025-08-26T09:40:18.000Z 46 715371 Not Buck Inc 2013 SUV 2025-09-03T11:22:09.000Z 47 GNN 071 Important Silva-Rich 2011 Sedan 2025-08-25T03:58:55.000Z 48 F22-KIU From Bass Inc 2011 Van 2025-09-02T18:18:20.000Z 49 3298 Than Jones, Lewis and Gutierrez 2019 Van 2025-09-15T05:15:59.000Z 50 7QR O68 Smile Blake, Stone and Ferguson 2010 Sedan 2025-08-24T04:00:11.000Z

In [0]:
#clean - upper case make col
df_veh = df_veh.withColumn("make",upper(col("make")))
display(df_veh)

vehicle_id license_plate model make year vehicle_type last_updated_timestamp 1 NXT-8646 Message FRANCIS, SMITH AND LEE 2023 Hatchback 2025-09-02T06:05:20.000Z 2 03S R43 Region LAWSON GROUP 2017 Sedan 2025-08-31T05:55:09.000Z 3 SKO H06 Prepare MORENO, RUIZ AND BARKER 2023 Luxury 2025-08-30T05:07:29.000Z 4 5R235 Pattern WELCH, MARTINEZ AND HENDRICKS 2019 Van 2025-09-09T05:52:10.000Z 5 925A Process RIVERA-ANDERSON 2014 Hatchback 2025-08-27T11:42:16.000Z 6 6-1130V On BROWN LTD 2014 SUV 2025-09-06T06:37:52.000Z 7 L25-TWN Plan GONZALEZ, RIOS AND RIOS 2020 Sedan 2025-09-15T13:14:48.000Z 8 GWY 958 Month SMITH, MCKENZIE AND BULLOCK 2017 Sedan 2025-09-11T16:20:12.000Z 9 4FG 919 Speak RICE, BARNES AND HERNANDEZ 2019 SUV 2025-09-11T02:43:13.000Z 10 6FD 648 Religious SCHWARTZ AND SONS 2017 SUV 2025-09-11T19:10:01.000Z 11 0N 8334L Name EVANS-KING 2010 SUV 2025-09-10T19:40:04.000Z 12 3YH 743 So MARTINEZ AND SONS 2011 Sedan 2025-09-03T19:53:15.000Z 13 ULA0960 Event REED INC 2010 Sedan 2025-08-31T05:38:52.000Z 14 982MCB Road CHAVEZ GROUP 2014 Luxury 2025-08-25T11:03:53.000Z 15 0I 6R0MOA Item ROBBINS LLC 2015 Sedan 2025-08-20T16:17:49.000Z 16 6WJ63 Store JOHNSON-PRINCE 2017 Van 2025-09-07T09:41:02.000Z 17 542-VGUN East PEREZ, LEE AND HARRIS 2023 Sedan 2025-09-02T18:06:22.000Z 18 RLY 8125 Low LUNA, BRYANT AND RHODES 2019 Luxury 2025-09-19T17:46:03.000Z 19 JV-9841 Now LAWRENCE INC 2019 SUV 2025-08-24T06:13:32.000Z 20 9U 7I8WLE Own WEBB, FOWLER AND MURRAY 2018 Luxury 2025-08-29T14:59:19.000Z 21 JYI-465 Rule ALLEN AND SONS 2014 Sedan 2025-09-02T02:34:55.000Z 22 53-FE88 Face STEPHENSON-ROBINSON 2012 Hatchback 2025-08-30T05:46:20.000Z 23 5T WT867 Information COLLINS PLC 2013 Van 2025-09-09T17:04:46.000Z 24 96-4009B Question WALKER PLC 2023 Van 2025-08-28T20:18:45.000Z 25 ZMM 744 Upon ARELLANO-WILLIAMS 2016 Luxury 2025-08-27T22:33:12.000Z 26 KE 35607 Go OWENS PLC 2021 Hatchback 2025-09-07T07:47:40.000Z 27 602 CUA Blood BAKER AND SONS 2016 Van 2025-09-13T05:32:48.000Z 28 UCT-785 Build SAMPSON-HARRISON 2010 Luxury 2025-08-21T12:41:00.000Z 29 7W 2C1MWN But GARCIA-CRAIG 2015 Hatchback 2025-09-12T05:34:31.000Z 30 KBS5092 Care CARTER AND SONS 2022 Hatchback 2025-08-27T04:57:47.000Z 31 8ESI316 Enter MARTIN AND SONS 2023 Sedan 2025-09-09T11:06:50.000Z 32 VQC N88 Consumer BOYD-CRUZ 2016 Hatchback 2025-09-20T01:23:10.000Z 33 4596 PJ Sort BUTLER-SALINAS 2019 Sedan 2025-09-20T08:41:13.000Z 34 MQL 012 Kind COLLIER-BAKER 2016 SUV 2025-09-09T16:45:34.000Z 35 487 MVB South GREGORY, SANCHEZ AND LEE 2019 Sedan 2025-09-09T11:03:29.000Z 36 6TU5286 Help SUMMERS-FOSTER 2022 Van 2025-09-03T16:13:47.000Z 37 393 0841 Per PALMER LTD 2020 SUV 2025-09-07T09:56:14.000Z 38 1W 5D5FFZ Mission HILL PLC 2015 Sedan 2025-09-16T17:24:54.000Z 39 2V 58100 Well WOODS-PARKS 2018 Sedan 2025-08-22T09:11:03.000Z 40 FGE 827 Expert PARSONS INC 2011 Hatchback 2025-09-16T16:04:50.000Z 41 LO 3414 Drop COLE LTD 2021 Luxury 2025-09-14T11:29:26.000Z 42 LB-9270 Government GUTIERREZ, MORENO AND DAVIS 2023 SUV 2025-08-27T09:52:46.000Z 43 6ID56 Call LEWIS PLC 2018 SUV 2025-09-12T18:28:24.000Z 44 094 TEH Court WARNER LLC 2020 SUV 2025-08-23T13:57:59.000Z 45 AWZ2518 With ROTH-RICH 2016 Hatchback 2025-08-26T09:40:18.000Z 46 715371 Not BUCK INC 2013 SUV 2025-09-03T11:22:09.000Z 47 GNN 071 Important SILVA-RICH 2011 Sedan 2025-08-25T03:58:55.000Z 48 F22-KIU From BASS INC 2011 Van 2025-09-02T18:18:20.000Z 49 3298 Than JONES, LEWIS AND GUTIERREZ 2019 Van 2025-09-15T05:15:59.000Z 50 7QR O68 Smile BLAKE, STONE AND FERGUSON 2010 Sedan 2025-08-24T04:00:11.000Z

In [0]:
#apply transfromations
vehicle_obj = transformations()
df_veh = vehicle_obj.dedup(df_veh,['vehicle_id'],'last_updated_timestamp')
df_veh = vehicle_obj.process_timestamp(df_veh)

In [0]:
if not spark.catalog.tableExists("pysparkdbt.silver.vehicles"):
    df_veh.write.format("delta")\
        .mode("append")\
        .saveAsTable("pysparkdbt.silver.vehicles")
else:
    vehicle_obj.upsert(spark, df_veh, ['vehicle_id'], 'vehicles', 'last_updated_timestamp')

In [0]:
%sql
SELECT COUNT(*) FROM pysparkdbt.silver.vehicles

COUNT(*) 50